# Supervised Learning: Classification

### Acknowledgements: Usman Alim 



Further Reading:

* `scikit-learn`: [user guide](https://scikit-learn.org/stable/user_guide.html).


## Quick Overview of scikit-learn

- [`scikit-learn`](https://scikit-learn.org/stable/) is the main machine learning library in the Python data science ecosystem.
- Implements many supervised (classification, regression) and unsupervised (clustering, density estimation, dimensionality reduction) learning algorithms.
- Relies heavily on `numpy`. Inputs and outputs are numpy arrays.
- Input data are expected to be $n \times D$ numerical arrays where $n$ is the number of observations, and $D$ is the number of features.
- Some feature wrangling may be needed, provides methods for feature extraction. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data processing related imports
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# model related imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

In [ ]:
# Processing data for model

import string
import re

def preprocess_text(raw_tweet):
    # Remove punctuation
    new_string = []
    for word in raw_tweet:
        # Check if the word is a URL
        if "https://" in word or "http://" in word:
            new_string.append(word)
        else:
            word = re.sub(r"[^A-Za-z0-9.\-]", " ", word)
            new_string.append(word)

    # Join the list of words into a string
    string1 = "".join(new_string)
    
    # Replace multiple spaces with a single space
    string1 = re.sub(r"\s+", " ", string1)
    
#     # Remove leading and trailing spaces
#     string1 = string1.strip()
    
    # Convert to lowercase
    string1 = string1.lower()

    return string1

p_data['preprocessed_text'] = p_data['Tweet'].apply(preprocess_text)
p_data

In [2]:
train_data = pd.read_csv('train80.csv')
test_data = pd.read_csv('test20.csv')


In [3]:
train_data['followers'] = train_data['followers'].fillna(-1)
test_data['followers'] = test_data['followers'].fillna(-1)

In [ ]:
Get the feature('Tweet') and lable ('Type') data from a Training and Test dataset
We use 80% of the dataset to train a model, and use the rest to test the model predictions

X_train, X_test, y_train, y_test = train_test_split(p_data['preprocessed_text'], 
                                                    p_data['Type'], test_size=.20, random_state=70)

# random_state = 70 gave us the best accuracy so far. tried varies mubers like  30, 50, 60, 80, 90

# splitting data and writing into csv file for consistent outputs. 



X_train, X_test = train_test_split(p_data, test_size=.20, random_state=70)

X_train.to_csv('train80.csv', index=False)
X_test.to_csv('test20.csv', index=False)


#Start from here:::::::::::

In [39]:
def get_c_vec_test_train(train_feature, test_feature,c_vec):

    # Fit and transform the preprocessed text
    c_vec_train = c_vec.fit_transform(train_feature)
    c_vect_test = c_vec.transform(test_feature)
    
    return c_vec_train, c_vect_test

def get_tf_vec_test_train(train_feature, test_feature, tfid_vec):

    tf_vec_train = tfid_vec.fit_transform(train_feature)
    tf_vec_test = tfid_vec.transform(test_feature)

    return tf_vec_train, tf_vec_test

def get_scalar_test_train(train_feature, test_feature, scaler ):
    train_feature = np.array(train_feature)
    test_feature = np.array(test_feature)
    
    train_feature = train_feature.reshape(-1, 1)
    test_feature = test_feature.reshape(-1, 1)
    
    train_scaled = scaler.fit_transform(train_feature)
    test_scaled = scaler.transform(test_feature)
    
    return  train_scaled, test_scaled
    
                                    
def get_NB(train_feature, test_feature, v_type, vectorizor, model):
     
    y_train = train_data['Type']
    y_test = test_data['Type']
    vectype = v_type
    
    vec_train = train_feature
    vect_test  = test_feature
    
    if (vectype == "c_vec"):
        vec_train, vect_test = get_c_vec_test_train(train_feature, test_feature, vectorizor)
    elif(vectype == "tfid_vec"):
        vec_train, vect_test =  get_tf_vec_test_train(train_feature, test_feature, vectorizor)                         
    else:
        print("Error with vector type")
     
        
     # Train a Naive Bayes classifier on the count vectorized text
    model.fit(vec_train, y_train)

    # Use the trained classifier to make predictions on the vectorized new text data
    pred_y = model.predict(vect_test)
    
    
    # Compute the accuracy score of the predicted labels
    accuracy = accuracy_score(y_test, pred_y)
    print("accuracy by NB using {} vectorizer: {:.4f}%".format(vectype, accuracy * 100))
    return pred_y
  
    
def get_SVC(train_feature, test_feature, v_type, vectorizor, model):
    
    y_train = train_data['Type']
    y_test = test_data['Type']
    
    vectype = v_type
    
    if (vectype == "c_vec"):
        vec_train, vect_test = get_c_vec_test_train(train_feature, test_feature,vectorizor)
        
    elif(vectype == "tfid_vec"):
        vec_train, vect_test =  get_tf_vec_test_train(train_feature, test_feature, vectorizor)
    
    elif(vectype == "scaler"):
        vec_train, vect_test =  get_scalar_test_train(train_feature, test_feature, vectorizor)
    else:
        print("Error with vector type")
    

    # Train SVM model using Tf vector

    clf.fit(vec_train, y_train)

    pred_y = clf.predict(vect_test)

    # Evaluate model
    accuracy = accuracy_score(y_test, pred_y)
    print("accuracy by SVC using {} vectorizer: {:.4f}%".format( vectype, accuracy * 100))

    return pred_y


def get_ConfusionMatrix(model, pred_y):
    
    y_test = test_data['Type']
        

    cm = confusion_matrix(y_test, pred_y)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    plt.show()
    

def get_ConMatrix_text(train_data, test_data):

    get_ConfusionMatrix(nb, 
                        get_NB(train_data, test_data,"tfid_vec", tfid_vec, nb) )

    get_ConfusionMatrix(clf,
                        get_SVC(train_data, test_data,"tfid_vec", tfid_vec, clf) )
    
    get_ConfusionMatrix(nb, 
                    get_NB(train_data, test_data,"c_vec", c_vec, nb) )

    get_ConfusionMatrix(clf,
                    get_SVC(train_data, test_data,"c_vec", c_vec, clf) )
    
    
def get_ConMatrix_num(train_data, test_data ):

    get_ConfusionMatrix(nb, 
                        get_NB(train_data, test_data,"", scaler, nb) )

    get_ConfusionMatrix(clf,
                        get_SVC(train_data, test_data,"scaler", scaler, clf) )


In [5]:
# Defining Naive Bayes classifier 
nb = MultinomialNB()

# Defining SVC model 
clf = SVC(kernel='linear', C=1, gamma='auto')

# Initialize the vectorizer with desired parameters
c_vec = CountVectorizer(max_features=1000)

# Vectorize tweets using TfidfVectorizer
tfid_vec = TfidfVectorizer()

# initialize the scaler 
scaler = StandardScaler()

In [7]:
def preprocess_numbers(df_num_col):

    # Convert the numbers to a string representation
    data_str = ["{:.4f}".format(x) for x in df_num_col]
    # document = " ".join(data_str)
    processed_num_list = data_str
    return processed_num_list